In [26]:
# neural network of many complex valued neurons in 3 layers
import numpy

In [27]:
class neuralNetwork:
    
    def __init__(self, inputnodes, hiddennodes, outputnodes, cats, periods):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes + 1 # increament for bias node
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.uniform(-1.0, 1.0, (self.hnodes, self.inodes))
        self.wih = numpy.array(self.wih, ndmin=2, dtype='complex128')
        self.wih += 1j * numpy.random.uniform(-1.0, 1.0, (self.hnodes, self.inodes))
        
        self.who = numpy.random.uniform(-1.0, 1.0, (self.onodes, self.hnodes))
        self.who = numpy.array(self.who, ndmin=2, dtype='complex128')
        self.who += 1j * numpy.random.random((self.onodes, self.hnodes))
        
        # number of output class categories
        self.categories = cats
        
        # todo periodicity
        self.periodicity = periods
        pass
    
    def z_to_class(self, z):
        # first work out the angle, but shift angle from [-pi/2, +pi.2] to [0,2pi]
        angle = numpy.mod(numpy.angle(z) + 2*numpy.pi, 2*numpy.pi)
        # from angle to category
        p = int(numpy.floor (self.categories * angle / (2*numpy.pi)))
        return p

    def class_to_angle(self, p):
        # class to angle, using bisector
        angle = ((p + 0.5) / self.categories) * 2 * numpy.pi
        return angle
    
    def status(self):
        print ("self.wih = ", self.wih)
        print ("self.who = ", self.who)
        pass

    def query(self, inputs_list):
        # add bias input
        inputs_list.append(1.0)
        
        # convert input to complex
        inputs = numpy.array(inputs_list, ndmin=2, dtype='complex128').T
        print("inputs = \n", inputs)
        
        # signal into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #print("hidden_inputs = ", hidden_inputs)
        #signal out of hidden layer
        hidden_outputs = numpy.exp(1j * numpy.angle(hidden_inputs))
        #print("hidden_outputs = ", hidden_outputs)
        
        # signal into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #print("final_input = ", final_inputs)
        #signal out of output layer
        final_outputs = numpy.exp(1j * numpy.angle(final_inputs))
        #print("final_outputs = ", final_outputs)
        
        # map to output classes
        output_classes = self.z_to_class(final_outputs)
        print("output_classes = ", output_classes)
        return output_classes
    
    def train(self, inputs_list, target_class_list):
        # add bias input
        inputs_list.append(1.0)
        
        # convert input to complex
        inputs = numpy.array(inputs_list, ndmin=2, dtype='complex128').T
        #print("inputs = \n", inputs)
        
        # map target classes to unit circle
        targets = numpy.exp(1j * self.class_to_angle(numpy.array(target_class_list, ndmin=2).T))
        #print("targets = \n", targets)

        # signal into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #print("hidden_inputs = ", hidden_inputs)
        #signal out of hidden layer
        hidden_outputs = numpy.exp(1j * numpy.angle(hidden_inputs))
        #print("hidden_outputs = ", hidden_outputs)
        
        # signal into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #print("final_inputs = ", final_inputs)
        #signal out of output layer
        final_outputs = numpy.exp(1j * numpy.angle(final_inputs))
        #print("final_outputs = ", final_outputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        #print("output_errors = ", output_errors)
        # hidden layer error is the output_errors split simply (not by weights)
        hidden_errors = numpy.dot(numpy.array(numpy.ones((self.hnodes)), ndmin=2).T / self.hnodes, output_errors)
        #print("hidden_errors = ", hidden_errors)
        
        # dw = e * x.T / (x.x.T)
        #dwho = (output_errors * numpy.conj(hidden_outputs.T)) / (self.hnodes)
        dwho = numpy.dot(output_errors, numpy.conj(hidden_outputs.T)) / (self.hnodes)
        #print("dwho = ", dwho)
        self.who += dwho
        
        #dwih = (hidden_errors * numpy.conj(inputs.T)) / (self.inodes)
        dwih = numpy.dot(hidden_errors, numpy.conj(inputs.T)) / (self.inodes)
        #print("dwih = ", dwih)
        self.wih += dwih        
    pass

In [28]:
# number of input, hidden and output nodes
input_nodes = 2
hidden_nodes = 2
output_nodes = 1

# categories, periodicity
categories = 2
periodicity = 1

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, categories, periodicity)
n.status()

self.wih =  [[-0.97560880-0.44299654j  0.39673175+0.45313716j -0.36348055-0.42098974j]
 [-0.78914030+0.38495572j  0.06405584+0.12566801j  0.93014655+0.4663918j ]]
self.who =  [[-0.15796713+0.42202923j -0.66723068+0.01251784j]]


In [29]:
# train neural network - OR

epochs = 5

for e in range(epochs):
    n.train([-1.0, -1.0], [0])
    n.train([-1.0, 1.0], [1])
    n.train([1.0, -1.0], [1])
    n.train([1.0, 1.0], [1])
    pass

In [30]:
# query after training
n.status()
n.query( [-1.0, -1.0] )
n.query( [-1.0, 1.0] )
n.query( [1.0, -1.0] )
n.query( [1.0, 1.0] )

self.wih =  [[-0.95393264-1.0836447j   0.39135471-1.35898696j -0.32501294-2.01430814j]
 [-0.76746414-0.25569244j  0.05867880-1.68645611j  0.96861417-1.1269266j ]]
self.who =  [[ 1.17035804+0.61847302j -0.13069239-0.41321431j]]
inputs = 
 [[-1.+0.j]
 [-1.+0.j]
 [ 1.+0.j]]
output_classes =  0
inputs = 
 [[-1.+0.j]
 [ 1.+0.j]
 [ 1.+0.j]]
output_classes =  1
inputs = 
 [[ 1.+0.j]
 [-1.+0.j]
 [ 1.+0.j]]
output_classes =  1
inputs = 
 [[ 1.+0.j]
 [ 1.+0.j]
 [ 1.+0.j]]
output_classes =  1


1